In [7]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch
from tqdm.notebook import tqdm
from PyPDF2 import PdfReader
from tqdm.notebook import tqdm
from textblob import TextBlob
import numpy as np
import spacy  
import re
from collections import Counter
from nltk.corpus import wordnet
import nltk
nltk.download('omw-1.4')
import pandas as pd
import numpy as np
from transformers import pipeline
import pdfplumber
import os
from difflib import SequenceMatcher
import pickle
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.nn.functional import softmax
import torch

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Load pdf text and headings from the pickle file
pdf_texts = pickle.load(open("pdf_texts.pkl", "rb"))
pdf_headings = pickle.load(open("pdf_headings.pkl", "rb"))

In [4]:
#loading spacy
nlp = spacy.load("en_core_web_sm")

# tokenizing the reports
tokenized_reports = {}

# Loop through each report in pdf_texts
for report_name, report_text in pdf_texts.items():
    # Use the nlp.pipe method to tokenize the report_text
    tokenized_report = [doc for doc in nlp.pipe([report_text])]
    # Add tokenized_report to tokenized_reports
    tokenized_reports[report_name] = tokenized_report

In [5]:
def find_related_financial_keywords(tokenized_reports):
    # financial_terms.txt is a list of financial keywords from Tilburg University
    with open('financial_terms.txt', 'r') as f:
        financial_keywords = [line.strip() for line in f]

    # Find all synsets related to financial keywords
    financial_synsets = [wordnet.synsets(keyword) for keyword in financial_keywords]

    # Flatten the list of synsets
    financial_synsets = [synset for sublist in financial_synsets for synset in sublist]

    # Find all lemmas for these synsets
    financial_lemmas = [lemma.name() for synset in financial_synsets for lemma in synset.lemmas()]

    # Initialize a dictionary to store the most frequent financial keywords for each document
    financial_keywords_counts_dict = {}

    # Iterate over each document in tokenized_reports
    for report_name, tokenized_report in tokenized_reports.items():

        # For each tokenized report, get a list of all tokens
        all_tokens = [token.text for doc in tokenized_report for token in doc]

        # Filter tokens to only include those in the financial lemmas list
        financial_tokens = [token for token in all_tokens if token in financial_lemmas]

        # Use Counter to count each financial token's occurrences
        financial_token_counts = Counter(financial_tokens)

        # Only keep the top 10 most frequent financial words
        top_10_financial_words = financial_token_counts.most_common(10)

        # Add the result to the dictionary
        financial_keywords_counts_dict[report_name] = top_10_financial_words

    return financial_keywords_counts_dict

# Call the function
financial_keywords_counts = find_related_financial_keywords(tokenized_reports)

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

def find_top10_frequent_words(tokenized_reports):
    # Select the two reports
    reports = ['FINAL-Q1-23-Shareholder-Letter', 'FINAL-Q2-23-Shareholder-Letter']

    # Initialize a dictionary to store the most frequent words for each selected report
    frequent_words_counts_dict = {}

    # Iterate over each selected report
    for report_name in reports:
        if report_name in tokenized_reports:
            tokenized_report = tokenized_reports[report_name]

            # For each tokenized report, get a list of all tokens
            all_tokens = [token.text.lower() for doc in tokenized_report for token in doc]

            # Filter out punctuation, stop words, and other non-alphabetic tokens
            all_tokens = [token for token in all_tokens if token.isalpha() and not nlp.vocab[token].is_stop]

            # Use Counter to count each token's occurrences
            token_counts = Counter(all_tokens)

            # Only keep the top 10 most frequent words
            top_10_words = token_counts.most_common(10)

            # Add the result to the dictionary
            frequent_words_counts_dict[report_name] = top_10_words

    return frequent_words_counts_dict

# Call the function
frequent_words_counts = find_top10_frequent_words(tokenized_reports)

# # Print the results
# for report_name, frequent_words_counts in frequent_words_counts.items():
#     print(f"Report name: {report_name}")
#     for word, count in frequent_words_counts:
#         print(f"Word: {word}, Count: {count}")
#     print("\n")

In [10]:
# Initialize a dictionary to store the joined sentences for each report.
joined_sentences = {}

# Iterate over each report in pdf_texts.
for report_name, report_text in pdf_texts.items():

    # Split the report text into sentences.
    sentences = nlp(report_text).sents

    # Initialize a list to hold the tokenized sentences for this report.
    tokenized_report_sentences = []

    # Iterate over each sentence.
    for sentence in sentences:
        # Tokenize, lemmatize, and remove stop words and punctuation.
        tokenized = [token.lemma_ for token in sentence if not token.is_stop and not token.is_punct]
        # Add the tokenized sentence to the list.
        tokenized_report_sentences.append(tokenized)

    # Join each tokenized sentence into a single string, and store them in a list.
    joined_report_sentences = [' '.join(sentence) for sentence in tokenized_report_sentences]

    # Add the joined sentences for this report to joined_sentences.
    joined_sentences[report_name] = joined_report_sentences

# # Print the joined sentences for each report.
# for report_name, joined_report_sentences in joined_sentences.items():
#     print(f"Report name: {report_name}")
#     print(joined_report_sentences)
#     print("\n")

In [8]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.nn.functional import softmax

def analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords):
      
    # Initialize a dictionary to store the sentences and their sentiment scores for each report
    sentiment_results_dict = {}

    # Iterate over each report
    for report_name, sentences in joined_sentences.items():
        # Initialize a dictionary to store the sentiment analysis results for the current report
        report_sentiment_results = {keyword: [] for keyword in keywords}

        # Create a list to hold sentence chunks
        sentence_chunks = []

        for sentence in sentences:
            # If a sentence exceeds 512 tokens, break it into chunks
            if len(sentence) >= 512:
                chunked_sentences = [sentence[i:i + 512] for i in range(0, len(sentence), 512)]
                sentence_chunks.extend(chunked_sentences)
            else:
                sentence_chunks.append(sentence)

        # Analyze the sentiment for each sentence chunk using BERT model
        for chunk in sentence_chunks:
            chunk_lower = chunk.lower()
            for keyword in keywords:
                if keyword in chunk_lower:
                    inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True, max_length=512)
                    outputs = model(**inputs)
                    probs = softmax(outputs.logits, dim=1)
                    sentiment_result = {
                        "label": "positive" if probs[0][1] > probs[0][0] else "negative",
                        "score": probs[0][1].item()
                    }
                    report_sentiment_results[keyword].append(sentiment_result)

        # Add the results to the dictionary
        sentiment_results_dict[report_name] = report_sentiment_results

    return sentiment_results_dict

keywords = ['revenue', 'forecast', 'profit']

# Call the function
sentences_with_keywords_and_sentiment = analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords)
print(sentences_with_keywords_and_sentiment)


{'COMBINED-Q4-17-Shareholder-Letter-FINAL': {'revenue': [{'label': 'positive', 'score': 0.5112976431846619}, {'label': 'positive', 'score': 0.5125821828842163}, {'label': 'positive', 'score': 0.5100852251052856}, {'label': 'positive', 'score': 0.5135365724563599}, {'label': 'positive', 'score': 0.5124308466911316}, {'label': 'positive', 'score': 0.5144804120063782}, {'label': 'positive', 'score': 0.5153369307518005}, {'label': 'positive', 'score': 0.5065018534660339}, {'label': 'positive', 'score': 0.5130241513252258}, {'label': 'positive', 'score': 0.5140935778617859}, {'label': 'positive', 'score': 0.5137550234794617}, {'label': 'positive', 'score': 0.5111774206161499}, {'label': 'positive', 'score': 0.5097774863243103}, {'label': 'positive', 'score': 0.5136916041374207}], 'forecast': [{'label': 'positive', 'score': 0.5105041861534119}, {'label': 'positive', 'score': 0.5136939883232117}, {'label': 'positive', 'score': 0.5156536102294922}, {'label': 'positive', 'score': 0.511071503162

In [12]:
# Initialize a dictionary to hold total scores for each keyword in each report
total_scores = {report: {keyword: 0 for keyword in keywords} for report in sentences_with_keywords_and_sentiment.keys()}

# Calculate total scores for each keyword in each report
for report_name, keywords_dict in sentences_with_keywords_and_sentiment.items():
    for keyword, sentiments in keywords_dict.items():
        for sentiment in sentiments:
            # If the sentiment is POSITIVE, add the score
            # If the sentiment is NEGATIVE, subtract the score
            if sentiment['label'] == 'POSITIVE':
                total_scores[report_name][keyword] += sentiment['score']
            else:
                total_scores[report_name][keyword] -= sentiment['score']

# Convert the total_scores to a DataFrame
df = pd.DataFrame(total_scores).T
df.reset_index(inplace=True)
df.columns = ['report_name', 'revenue_score', 'forecast_score', 'profit_score']

In [13]:
df

,report_name,revenue_score,forecast_score,profit_score
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,-7.171771,-4.104856,-5.114237
1,FINAL-Q1-18-Shareholder-Letter,-2.048475,-3.074118,0.000000
2,FINAL-Q1-19-Shareholder-Letter,-2.556345,-2.053208,0.000000
3,FINAL-Q1-20-Shareholder-Letter,-3.085288,-3.080612,-1.534656
4,FINAL-Q1-21-Shareholder-Letter,-2.560761,-4.100819,-0.512070
5,FINAL-Q1-22-Shareholder-Letter,-7.671927,-4.108910,-2.043929
6,Final-Q1-23-Shareholder-Letter,-12.309192,-4.617841,-3.077486
7,FINAL-Q2-18-Shareholder-Letter,-7.166668,-5.632038,-2.553709
8,FINAL-Q2-20-Shareholder-Letter-V3-with-Tables,-3.075913,-3.593011,0.000000
9,FINAL-Q2-21-Shareholder-Letter,-3.063522,-4.101212,-1.543377


In [14]:
# Add a new column 'total_score'
df['total_score'] = df['revenue_score'] + df['forecast_score'] + df['profit_score']

# Print the results
df

,report_name,revenue_score,forecast_score,profit_score,total_score
0,COMBINED-Q4-17-Shareholder-Letter-FINAL,-7.171771,-4.104856,-5.114237,-16.390864
1,FINAL-Q1-18-Shareholder-Letter,-2.048475,-3.074118,0.000000,-5.122592
2,FINAL-Q1-19-Shareholder-Letter,-2.556345,-2.053208,0.000000,-4.609553
3,FINAL-Q1-20-Shareholder-Letter,-3.085288,-3.080612,-1.534656,-7.700556
4,FINAL-Q1-21-Shareholder-Letter,-2.560761,-4.100819,-0.512070,-7.173651
5,FINAL-Q1-22-Shareholder-Letter,-7.671927,-4.108910,-2.043929,-13.824766
6,Final-Q1-23-Shareholder-Letter,-12.309192,-4.617841,-3.077486,-20.004519
7,FINAL-Q2-18-Shareholder-Letter,-7.166668,-5.632038,-2.553709,-15.352416
8,FINAL-Q2-20-Shareholder-Letter-V3-with-Tables,-3.075913,-3.593011,0.000000,-6.668923
9,FINAL-Q2-21-Shareholder-Letter,-3.063522,-4.101212,-1.543377,-8.708111
